In [100]:
import numpy as np
import sympy as sym

In [101]:
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [102]:
def GetHermite(n,x):
    
    if n==0:
        H=1
    elif n==1:
        H=2*x
    else:    
        H=(2*x*(GetHermite(n-1,x)))-(2*(n-1)*(GetHermite(n-2,x)))
    return H


In [103]:
print (GetHermite(3,x))

2*x*(4*x**2 - 2) - 8*x


In [104]:
def GetDHermite(n,x):
    H1 = GetHermite(n,x)
    return sym.diff(H1,x,1)

In [105]:
print (GetDHermite(3,x))

24*x**2 - 12


In [106]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [107]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [108]:
def GetAllRootsGHer(n):

    xn = np.linspace(-np.sqrt(4*n+1),np.sqrt(4*n+1),100)
    
    Hermite = []
    DHermite = []
    
    for i in range(n+1):
        Hermite.append(GetHermite(i,x))
        DHermite.append(GetDHermite(i,x))
    
    poly = sym.lambdify([x],Hermite[n],'numpy')
    Dpoly = sym.lambdify([x],DHermite[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [109]:
print (GetAllRootsGHer(2))

[-0.70710678  0.70710678]


In [110]:
def GetWeightsGHer(n):
    Roots = GetAllRootsGHer(n)
    Dleguerre=[]
    for i in range(n+1):
        Dleguerre.append(GetDHermite(i,x))
    
    Dpoly = sym.lambdify([x],Dleguerre[n],'numpy')
    Weights = ((2**(n-1))*(np.math.factorial(n))*(np.sqrt(np.pi)))/(((n)**2)*((GetHermite(n-1,Roots))**2))
    
    return Weights

In [111]:
raices = GetAllRootsGHer(8)
pesos = GetWeightsGHer(8)
funcion = lambda x: x**2
I = 0
for i in range(8):
    I += pesos[i]*funcion(raices[i])


In [112]:
I

0.8862269255175592